#### *Basic information on these data sets can be viewed in 'Data Summary.ipynb'*
#### Features inspired by others' kernel contributions*

In [1]:
import numpy as np
from scipy import stats
import pandas as pd
pd.set_option('precision',4)

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn')

import sys
print (sys.version)

2.7.14 |Anaconda, Inc.| (default, Oct 15 2017, 03:34:40) [MSC v.1500 64 bit (AMD64)]


In [2]:
# Combine order, products, and order_products_prior datasets
orders_df = pd.read_csv('input/orders.csv')
orders_df = orders_df[orders_df.eval_set=='prior']

prior_order_features_df = pd.read_csv('input/order_products__prior.csv')
prior_order_features_df = orders_df.merge(prior_order_features_df, how='left', on='order_id')

products_df = pd.read_csv('input/products.csv')
prior_order_features_df = prior_order_features_df.merge(products_df, how='left', on='product_id')

prior_order_features_df.sample(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
32326551,454738,205509,prior,34,3,19,30.0,1710,9,0,Kombucha Gingerade,31,7
23211624,2134050,147373,prior,2,3,17,30.0,24623,32,0,Chef Size Extra Virgin Olive Oil,19,13
7049447,2840960,44769,prior,38,3,10,8.0,19894,5,0,Enlightened Organic Raw Kombucha,31,7
27085187,3163388,172166,prior,4,3,8,18.0,4865,2,1,100% Cranberry Mango Juice,98,7
5540892,1429086,35227,prior,39,1,7,8.0,47156,20,1,Coconut Almond Unsweetened Creamer Blend,53,16
11960311,1214346,75705,prior,32,3,14,5.0,24852,9,1,Banana,24,4
26606451,1596867,169181,prior,2,6,19,13.0,42016,1,1,Non GMO Organic Brown Eggs,86,16
11295809,3042051,71565,prior,7,6,11,9.0,45288,27,1,Honey Smoked Turkey,96,20
25145823,1749875,159641,prior,2,0,12,5.0,21903,16,0,Organic Baby Spinach,123,4
7727762,688786,49063,prior,4,6,18,1.0,37275,1,0,Sofia Blanc De Blancs,134,5


In [3]:
prior_order_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32434489 entries, 0 to 32434488
Data columns (total 13 columns):
order_id                  int64
user_id                   int64
eval_set                  object
order_number              int64
order_dow                 int64
order_hour_of_day         int64
days_since_prior_order    float64
product_id                int64
add_to_cart_order         int64
reordered                 int64
product_name              object
aisle_id                  int64
department_id             int64
dtypes: float64(1), int64(10), object(2)
memory usage: 3.4+ GB


In [4]:
# check for null or nan values
prior_order_features_df.isnull().sum()

order_id                        0
user_id                         0
eval_set                        0
order_number                    0
order_dow                       0
order_hour_of_day               0
days_since_prior_order    2078068
product_id                      0
add_to_cart_order               0
reordered                       0
product_name                    0
aisle_id                        0
department_id                   0
dtype: int64

It makes since that 'days_since_prior_order' would be null for users' initial orders

In [5]:
# Drop non-numeric fields
prior_order_features_df.drop(['eval_set','product_name'],axis=1, inplace=True)

# Drop initial customer orders and generate sample for evaluation
prior_order_sample_df = prior_order_features_df.dropna().sample(frac=0.01)

In [15]:
# Descriptive statistics of sample
prior_order_sample_df.drop(['order_id','user_id','product_id'],axis=1).describe()

,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,aisle_id,department_id
count,303564.0000,303564.0000,303564.0000,303564.0000,303564.0000,303564.0000,303564.000,303564.0000
mean,18.2416,2.7398,13.4110,11.1313,8.3870,0.6291,71.217,9.9294
std,17.5823,2.0878,4.2541,8.7849,7.1397,0.4830,38.242,6.2831
min,2.0000,0.0000,0.0000,0.0000,1.0000,0.0000,1.000,1.0000
25%,6.0000,1.0000,10.0000,5.0000,3.0000,0.0000,31.000,4.0000
50%,12.0000,3.0000,13.0000,8.0000,6.0000,1.0000,83.000,9.0000
75%,25.0000,5.0000,16.0000,15.0000,12.0000,1.0000,107.000,16.0000
max,99.0000,6.0000,23.0000,30.0000,111.0000,1.0000,134.000,21.0000


In [7]:
# Evaluate correlation among raw features 
prior_order_sample_df.drop(['order_id','user_id','reordered'],axis=1).corr().style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,aisle_id,department_id
order_number,1.0,0.016,-0.04,-0.36,-0.0021,-0.0037,0.0051,0.0042
order_dow,0.016,1.0,0.014,-0.03,-0.0058,-0.0087,0.00057,0.0062
order_hour_of_day,-0.04,0.014,1.0,0.0017,0.00061,-0.015,-0.0036,-0.0097
days_since_prior_order,-0.36,-0.03,0.0017,1.0,0.002,0.053,0.0051,-0.00013
product_id,-0.0021,-0.0058,0.00061,0.002,1.0,0.0056,0.0033,-0.028
add_to_cart_order,-0.0037,-0.0087,-0.015,0.053,0.0056,1.0,0.0076,0.028
aisle_id,0.0051,0.00057,-0.0036,0.0051,0.0033,0.0076,1.0,0.063
department_id,0.0042,0.0062,-0.0097,-0.00013,-0.028,0.028,0.063,1.0


It looks like 'order_dow' and 'order_hour_of_day' as well as 'add_to_cart_order' and 'days_since_prior_order' are the highest correlated pairs of features. 

In [8]:
# Utilize RFC to evaluate raw feature predictive importance
from sklearn.ensemble import RandomForestClassifier

y = prior_order_sample_df['reordered']
X = prior_order_sample_df.drop(['reordered'],axis=1)

cls = RandomForestClassifier(n_estimators=50)
cls.fit(X, y)

features = X.columns
feature_importances = cls.feature_importances_

features_df = pd.DataFrame({'features': features, 'importance': feature_importances})
features_df.sort_values('importance', inplace=True, ascending=False)

features_df.head()

,features,importance
2,order_number,0.1618
1,user_id,0.1360
0,order_id,0.1355
6,product_id,0.1285
5,days_since_prior_order,0.0871


In [9]:
# Let's try this again leaving out the irrelevant features
from sklearn.ensemble import RandomForestClassifier

y = prior_order_sample_df['reordered']
X = prior_order_sample_df.drop(['reordered','order_number','order_id','user_id'],axis=1)

cls = RandomForestClassifier(n_estimators=50)
cls.fit(X, y)

features = X.columns
feature_importances = cls.feature_importances_

features_df = pd.DataFrame({'features': features, 'importance': feature_importances})
features_df.sort_values('importance', inplace=True, ascending=False)

features_df.head()

,features,importance
3,product_id,0.2752
1,order_hour_of_day,0.1689
2,days_since_prior_order,0.1603
4,add_to_cart_order,0.1335
0,order_dow,0.1061


These results are more interesting and will inform feature development

### Create User Order Features for Prior Orders

In [10]:
# develop features to evaluate for predictive ability
# User order features
prior_order_features_df['user_orders'] = prior_order_features_df.groupby('user_id')['order_id'].transform('count')
prior_order_features_df['basket_size'] = prior_order_features_df.groupby(['user_id','order_id'])['product_id'].transform('count')
prior_order_features_df['avg_basket_size'] = prior_order_features_df.groupby('user_id')['basket_size'].transform(np.mean)
prior_order_features_df['avg_reorders_per_basket'] = ( prior_order_features_df.groupby(['user_id'])['reordered'].transform(np.mean) / prior_order_features_df.avg_basket_size)
prior_order_features_df['user_order_dow_mode'] = prior_order_features_df.groupby('user_id')['order_dow'].transform(lambda x: stats.mode(x)[0])
prior_order_features_df['user_order_hour_mode'] = prior_order_features_df.groupby('user_id')['order_hour_of_day'].transform(lambda x: stats.mode(x)[0])
prior_order_features_df['avg_days_between_orders'] = prior_order_features_df.groupby('user_id')['days_since_prior_order'].transform(np.mean)
# User & Product features
prior_order_features_df['avg_add_to_cart_order'] = prior_order_features_df.groupby(['user_id','product_id'])['add_to_cart_order'].transform(np.mean)
prior_order_features_df['user_prod_reorders'] = prior_order_features_df.groupby(['user_id','product_id'])['reordered'].transform(np.sum)
prior_order_features_df['user_prod_reorder_rate'] = ( prior_order_features_df.user_prod_reorders / prior_order_features_df.user_orders )
# Product features
prior_order_features_df['prod_order_count'] = prior_order_features_df.groupby('product_id')['add_to_cart_order'].transform('count')
prior_order_features_df['prod_reorders'] = prior_order_features_df.groupby('product_id')['reordered'].transform(np.sum)
prior_order_features_df['prod_reorder_rate'] = ( prior_order_features_df.prod_reorders / prior_order_features_df.prod_order_count )

In [14]:
prior_order_features_df.head(10)

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,aisle_id,...,avg_reorders_per_basket,user_order_dow_mode,user_order_hour_mode,avg_days_between_orders,avg_add_to_cart_order,user_prod_reorders,user_prod_reorder_rate,prod_order_count,prod_reorders,prod_reorder_rate
0,2539329,1,1,2,8,NaN,196,1,0,77,...,0.1111,4,7,20.2593,1.4000,9,0.1525,35791,27791,0.7765
1,2539329,1,1,2,8,NaN,14084,2,0,91,...,0.1111,4,7,20.2593,2.0000,0,0.0000,15935,12923,0.8110
2,2539329,1,1,2,8,NaN,12427,3,0,23,...,0.1111,4,7,20.2593,3.3000,9,0.1525,6476,4797,0.7407
3,2539329,1,1,2,8,NaN,26088,4,0,23,...,0.1111,4,7,20.2593,4.5000,1,0.0169,2523,1360,0.5390
4,2539329,1,1,2,8,NaN,26405,5,0,54,...,0.1111,4,7,20.2593,5.0000,1,0.0169,1214,536,0.4415
5,2398795,1,2,3,7,15.0,196,1,1,77,...,0.1111,4,7,20.2593,1.4000,9,0.1525,35791,27791,0.7765
6,2398795,1,2,3,7,15.0,10258,2,0,117,...,0.1111,4,7,20.2593,3.3333,8,0.1356,1946,1389,0.7138
7,2398795,1,2,3,7,15.0,12427,3,1,23,...,0.1111,4,7,20.2593,3.3000,9,0.1525,6476,4797,0.7407
8,2398795,1,2,3,7,15.0,13176,4,0,24,...,0.1111,4,7,20.2593,6.0000,1,0.0169,379450,315913,0.8326
9,2398795,1,2,3,7,15.0,26088,5,1,23,...,0.1111,4,7,20.2593,4.5000,1,0.0169,2523,1360,0.5390


In [17]:
# confirm record count
prior_order_features_df.shape[0]

32434489

In [18]:
# Write full set to file
prior_order_features_df.to_csv('transformed/prior_order_features.csv')

In [25]:
# Generate sample for evaluation
prior_order_sample_df = prior_order_features_df.dropna().sample(frac=0.03)

In [26]:
prior_order_sample_df.shape[0]

910693

In [27]:
# Evaluate correlation among generated features 
prior_order_sample_df.drop(['order_id','user_id','order_number','reordered'],axis=1).corr().style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,aisle_id,department_id,user_orders,basket_size,avg_basket_size,avg_reorders_per_basket,user_order_dow_mode,user_order_hour_mode,avg_days_between_orders,avg_add_to_cart_order,user_prod_reorders,user_prod_reorder_rate,prod_order_count,prod_reorders,prod_reorder_rate
order_dow,1.0,0.013,-0.031,-0.0023,-0.0086,-0.0032,0.0048,0.0041,-0.012,-0.02,0.02,0.28,0.0082,-0.027,-0.011,0.0054,0.0047,-0.015,-0.014,-0.019
order_hour_of_day,0.013,1.0,0.0043,0.00051,-0.015,-0.0017,-0.008,-0.04,-0.023,-0.0083,-0.029,0.0072,0.31,0.044,-0.011,-0.035,-0.016,-0.0017,-0.0018,-0.013
days_since_prior_order,-0.031,0.0043,1.0,3.6e-05,0.052,0.0053,0.00027,-0.37,0.079,0.035,-0.17,-0.0033,0.05,0.67,0.041,-0.23,-0.002,-0.019,-0.019,-0.035
product_id,-0.0023,0.00051,3.6e-05,1.0,0.0047,0.0024,-0.032,0.002,0.0071,0.0069,-0.014,6.9e-05,0.0013,0.00097,0.0054,-0.0077,-0.014,0.013,0.0048,0.012
add_to_cart_order,-0.0086,-0.015,0.052,0.0047,1.0,0.009,0.028,0.25,0.67,0.51,-0.3,-0.019,-0.0062,0.029,0.83,-0.14,-0.26,-0.092,-0.094,-0.14
aisle_id,-0.0032,-0.0017,0.0053,0.0024,0.009,1.0,0.062,0.0076,0.014,0.014,-0.019,-0.0029,0.0023,0.00037,0.011,-0.0021,-0.0066,-0.22,-0.22,0.014
department_id,0.0048,-0.008,0.00027,-0.032,0.028,0.062,1.0,0.019,0.024,0.025,-0.011,0.0022,-0.012,-0.0017,0.035,-0.03,-0.036,-0.3,-0.28,-0.13
user_orders,0.0041,-0.04,-0.37,0.002,0.25,0.0076,0.019,1.0,0.38,0.49,-0.11,-0.039,-0.073,-0.55,0.31,0.42,-0.14,0.017,0.016,0.065
basket_size,-0.012,-0.023,0.079,0.0071,0.67,0.014,0.024,0.38,1.0,0.77,-0.44,-0.028,-0.011,0.042,0.66,-0.012,-0.26,-0.03,-0.032,-0.041
avg_basket_size,-0.02,-0.0083,0.035,0.0069,0.51,0.014,0.025,0.49,0.77,1.0,-0.58,-0.037,-0.014,0.054,0.62,0.044,-0.28,-0.018,-0.019,-0.016


We see a high correlation between default features and their related engineered features.

In [28]:
# check ratio of reordered products vs non-reordered products 'Null Accuracy Check'
prior_order_features_df.reordered.value_counts(normalize=True)

1    0.5897
0    0.4103
Name: reordered, dtype: float64

In [29]:
# check null accuracy of sample set
prior_order_sample_df.reordered.value_counts(normalize=True)

1    0.6302
0    0.3698
Name: reordered, dtype: float64

It looks like the balance of reorders in the sample is slightly higher than the original.

### Evaluate Feature Importance

In [30]:
# Utilize RFC to evaluate feature predictive importance
from sklearn.ensemble import RandomForestClassifier

y = prior_order_sample_df['reordered']
X = prior_order_sample_df.drop(['reordered','order_id', 'user_id','order_number'],axis=1)

cls = RandomForestClassifier(n_estimators=50)
cls.fit(X, y)

features = X.columns
feature_importances = cls.feature_importances_

features_df = pd.DataFrame({'features': features, 'importance': feature_importances})
features_df.sort_values('importance', inplace=True, ascending=False)

features_df.head()

,features,importance
15,user_prod_reorders,0.3067
16,user_prod_reorder_rate,0.2529
19,prod_reorder_rate,0.0428
7,user_orders,0.0399
10,avg_reorders_per_basket,0.0321


We now have our top 5 features of significance using RFC but will leave out 'user_prod_reorders' due to correlation with 'user_prod_reoder_rate' and 'user_orders' since its not a relevant count.

In [31]:
# Write sample set to file
prior_order_sample_df.to_csv('transformed/prior_order_features_sample.csv')

### Add prior order features to training data for validation

In [44]:
# create validation data set, keeping features of importance
train_order_features_df = pd.read_csv("input/order_products__train.csv") 
orders_df = pd.read_csv('input/orders.csv')
train_order_features_df = train_order_features_df.merge(orders_df, how='left', on='order_id')
train_order_features_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,49302,1,1,112108,train,4,4,10,9.0
1,1,11109,2,1,112108,train,4,4,10,9.0
2,1,10246,3,0,112108,train,4,4,10,9.0
3,1,49683,4,0,112108,train,4,4,10,9.0
4,1,43633,5,1,112108,train,4,4,10,9.0


In [45]:
train_order_features_df.drop(['eval_set','order_number'],axis=1,inplace=True)

In [46]:
temp_prior_orders_df = prior_order_features_df[['user_id', 'avg_reorders_per_basket', 'avg_basket_size', 'avg_days_between_orders']].drop_duplicates()

train_order_features_df = train_order_features_df.merge(temp_prior_orders_df, how='left', on='user_id')

In [47]:
temp_prior_orders_df = prior_order_features_df[['product_id', 'prod_reorder_rate']].drop_duplicates()

train_order_features_df = train_order_features_df.merge(temp_prior_orders_df, how='left', on='product_id')

In [48]:
temp_prior_orders_df = prior_order_features_df[['user_id','product_id','avg_add_to_cart_order','user_prod_reorder_rate','user_prod_reorders']].drop_duplicates()

train_order_features_df = train_order_features_df.merge(temp_prior_orders_df, how='left', on=['user_id','product_id'])

In [49]:
train_order_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1384617 entries, 0 to 1384616
Data columns (total 15 columns):
order_id                   1384617 non-null int64
product_id                 1384617 non-null int64
add_to_cart_order          1384617 non-null int64
reordered                  1384617 non-null int64
user_id                    1384617 non-null int64
order_dow                  1384617 non-null int64
order_hour_of_day          1384617 non-null int64
days_since_prior_order     1384617 non-null float64
avg_reorders_per_basket    1384617 non-null float64
avg_basket_size            1384617 non-null float64
avg_days_between_orders    1384617 non-null float64
prod_reorder_rate          1384608 non-null float64
avg_add_to_cart_order      828824 non-null float64
user_prod_reorder_rate     828824 non-null float64
user_prod_reorders         828824 non-null float64
dtypes: float64(8), int64(7)
memory usage: 169.0 MB


In [50]:
# check for null or nan values
train_order_features_df.isnull().sum()

order_id                        0
product_id                      0
add_to_cart_order               0
reordered                       0
user_id                         0
order_dow                       0
order_hour_of_day               0
days_since_prior_order          0
avg_reorders_per_basket         0
avg_basket_size                 0
avg_days_between_orders         0
prod_reorder_rate               9
avg_add_to_cart_order      555793
user_prod_reorder_rate     555793
user_prod_reorders         555793
dtype: int64

In [51]:
# replace NaN values with zero
train_order_features_df = train_order_features_df.fillna(0)
train_order_features_df.isnull().sum()

order_id                   0
product_id                 0
add_to_cart_order          0
reordered                  0
user_id                    0
order_dow                  0
order_hour_of_day          0
days_since_prior_order     0
avg_reorders_per_basket    0
avg_basket_size            0
avg_days_between_orders    0
prod_reorder_rate          0
avg_add_to_cart_order      0
user_prod_reorder_rate     0
user_prod_reorders         0
dtype: int64

In [52]:
# write to file
train_order_features_df.to_csv('transformed/train_order_features.csv')

In [53]:
# check ratio of reordered products vs non-reordered products 'Null Accuracy Check'
train_order_features_df.reordered.value_counts(normalize=True)

1    0.5986
0    0.4014
Name: reordered, dtype: float64

We see a similar reorder rate among the training data set from the prior order set.

In [54]:
# capture sample 
train_order_features_df = train_order_features_df.sample(frac=0.6)
train_order_features_df.to_csv('transformed/train_order_features_sample.csv')
train_order_features_df.shape[0]

830770

In [55]:
# compare sample reordered ratio
train_order_features_df.reordered.value_counts(normalize=True)

1    0.5988
0    0.4012
Name: reordered, dtype: float64